# Data used to perform the analysis
The precidency of Donald J. Trump began at noon EST (17:00 UTC) on January 20, 2017 when he was inaugurated as the 45th president of the United States, and will com to an end the on January 20, 2021 as he ultimately lost the 2020 presidential election to Joe Biden. It is not far fetched to say it has been bizare precidency compared to the most recent precidencies. 

It feels like America has been splitted in two the suporters of Trump and those agianst him - Repulicans agianst Democrats. In this project we wanted to explore if our hypothesis of polarization can be seen or rejected by analyzing the Congress of United States behavoir on the social media Twitter including the infamous Twitter account manged by Donald J. Trump. The idea was to analyze the congress tweets from the time of the 45th precidency to explore potential polarization.

With access to the Twitter API it is only possible to exctract the most reason 3200 tweets from a given account (3200 tweet do not go far back for many american polticians). However, Twitter's Terms of Service do allow for datasets of tweets ID's to be distributed to third parties (not the full JSON). Luckily we found two sources that keep tweet id open very related to our project and one sources that stored the full length tweets of Donald Trump namely:

* ** 115th U.S. Congress Tweet Ids: **
    A open dataset with 2,041,399 tweets from the Twitter accounts of members of the 115th U.S. Congress collected in the period of January 27, 2017 and January 2, 2019.  *Littman, Justin, 2017, "115th U.S. Congress Tweet Ids", https://doi.org/10.7910/DVN/UIVHQR, Harvard Dataverse, V5.*

* ** 116th U.S. Congress Tweet Ids **
    A open dataset with 2,817,747 tweets from the Twitter accounts of members of the 116th U.S. Congress collected in the period of January 27, 2019 and May 7, 2020  * Wrubel, Laura; Kerchner, Daniel, 2020, "116th U.S. Congress Tweet Ids", https://doi.org/10.7910/DVN/MBOJNS, Harvard Dataverse*

* ** Trump Twitter Archive **
    A site dedicated to scrape every single tweet from Donald J. Trump. Here we downloaded all tweets in the periods of January 27, 2017 and January 2, 2019 and January 27, 2019 and May 7, 2020. https://www.thetrumparchive.com/


Examing the Harvard Dataverse we discovered it contains mostly tweets from the American congress but unfortunally also a number of random profiles. Furthermore, many of tweets are associated with not just congress members but also profiles linked to for instance the different Caucuses, officals account on behalf of all Repulican Senate members etc. Furthermore it was not listed what party the account where associated with. However, we found a list of congress memebers with their party association, whether they act as Senator or Represenative and, most important for the project, their twitter profile (if they have an account the vast majority has.). Two different sources for the 115 and 116 congress have been utilized namely

* 116. Congress twitter info: (website) https://triagecancer.org/congressional-social-media 
* 115. Congress  twitter info : (PDF) https://www.sciencecoalition.org/wp-content/uploads/2018/09/115th-Congress-Twitter-Handles.pdf  



This notebook consist of three parts and one smaller part devoted to extrating Trump tweets  id (part 0). So in total:
* Part 0: Exstract Trump Tweet IDs
    Here the tweet id are exstacted from the tweets made publicly by https://www.thetrumparchive.com/
* Part 1: Hydrate Tweets
    In this part all the tweet ids from Harvard data archive as well as Trumps tweet id, are hydrated. I.e. the ids are turned back into tweets with metadata. 
* Part 2: Cleanup
    As the Harvard data archive hav 
* Part 3: Preprocess the twitter data

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

## Part 0: Exstract Trump Tweet IDs
As the site Trump Twitter Achive (https://www.thetrumparchive.com/) store Donald Trump's tweets in a different format than how it typically exstracted from API to combat this we exstracted 

In [ ]:
from src.data.trump_tweet_ids import get_trump_tweet_ids

In [4]:
df_trump_tweets1 = pd.read_csv('../Data/raw/tweets/trump_tweets_1st.csv')  
df_trump_tweets2 = pd.read_csv('../Data/raw/tweets/trump_tweets_1st.csv')
df_trump = pd.concat([df_trump_tweets1, df_trump_tweets2])

In [6]:
filepath = "../Data/raw/tweets/trump_id.txt"
get_trump_tweet_ids(df_trump, filepath)

11326 tweet ids saved


## Part 1: Hydrate Tweets
The process of turning tweet ID's into actual tweets with metadata is called *hydration* and requires Twitter delopper account https://developer.twitter.com/en/apply-for-access (which is typically pretty easy to acquire).

In [24]:
import tweepy
from src.tools.twitter_api_credentials import api_key, api_secret_key, access_token, access_token_secret
from src.data.hydrate import hydrate_tweets

In [26]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')


In [28]:
representatives115 = np.loadtxt(
    "../Data/Raw/Tweets/representatives115.txt", dtype=int
)
representatives116 = np.loadtxt(
    "../Data/Raw/Tweets/representatives116.txt", dtype=int
)
senators115 = np.loadtxt(
    "../Data/Raw/Tweets/senators115.txt", dtype=int
)
senators116 = np.loadtxt(
    "../Data/Raw/Tweets/senators116.txt", dtype=int
)
trump = np.loadtxt(
    "../Data/Raw/Tweets/trump_id.txt", dtype=int
)

Note running the cell below take $24 \pm 6$ hours as the twitter API set limits to how much can be exstracted.

In [ ]:
congress = np.concatenate([representatives115, representatives116, senators115, senators116, trump])
filepath = "../Data/interim/congress.pkl"
api = tweepy.API(auth, wait_on_rate_limit=True)

hydrate_tweets(
    tweet_ids=congress,
    filepath=filepath,
    api = api
)

## Part 2: Cleanup

In [33]:
congress = pd.read_pickle('../Data/Interim/congress.pkl')
twitter_handles = pd.read_table('../Data/Processed/Twitter_Handles_updated.csv', sep = ',')

s1 = set(twitter_handles['twitter_display_name'])
s2 = set(congress.user_name.unique())

In [38]:
non_overlapping_twitter_profiles = s1 ^ s2

In [ ]:
# Make sure tweets only comes from people that twitter handles exist for. 
congress = congress[congress.user_name.isin(s1)]

In [ ]:
# Keep only the periods from Harward:
mask = (
    #January 27, 2017 and January 2, 2019 
    (congress.created_at > '2017-1-27 00:00:00') & (congress.created_at < '2019-1-2 00:00:00')
    | 
    #January 27, 2019 and May 7, 2020 
    (congress.created_at > '2019-1-27 00:00:00') & (congress.created_at < '2020-5-7 00:00:00')
)
congress = congress[mask]

In [ ]:
congress = congress.drop_duplicates(keep='first')
congress = congress.sort_values(by='created_at')
congress = congress.reset_index(drop=True)
congress.to_pickle("Data/interim/congress_cleaned.pkl")

In [ ]:
# Extract the tweets ids and convert them to integers
ids = list(congress.id.astype(int).values)

filepath = "../Data/raw/tweets/Cleaned_tweet_id.txt"
with open(filepath, 'w') as output:
    for row in ids:
        output.write(str(row) + '\n')

    print(f'{len(ids)} tweet ids saved.')

### Part 2.1 Shortcut to exstract the data

The dataframe with tweets of from congress after cleanup contain 60 % rows. 

Note running the cell below take $10 \pm 2.5$ hours as the twitter API set limits to how much can be exstracted.

In [ ]:
congress_tweet_id = np.loadtxt("../Data/Raw/Tweets/Cleaned_tweet_id.txt", dtype=int)
filepath = "../Data/interim/congress_cleaned.pkl"

hydrate_tweets(
    tweet_ids=congress_tweet_id,
    filepath=filepath,
    api = api
)

## Part 3: Preprocess the twitter data

In [ ]:
congress = pd.read_pickle('../Data/Interim/congress_cleaned.pkl')

In [ ]:
special_characters = ",._´&’%':€$£!?#"
character_set = {
    "characters": "abcdefghijklmnopqrstuvwxyz0123456789" + special_characters,
    "space": " ",
}
alphabet = "".join(character_set.values())

In [ ]:
regex_links = re.compile("http\S+")
regex_whitespace = re.compile("[\s|-]+")
regex_unknown = re.compile(f"[^{alphabet}]+")

In [ ]:
regex_html_tags = {
    "&amp": "and",
    "&lt": "<",
    "&gt": ">",
    "&quot": '"',
    "&apos": "'",
}

In [ ]:
## Replace unicode charetars
for pattern_string, char in regex_html_tags.items():
    congress_tweets["text"] = congress_tweets["text"].str.replace(pattern_string, char)

In [ ]:
congress_tweets["text"] = (congress_tweets["text"]
    .str.lower()
    .str.replace(regex_links, "")
    .str.replace(regex_whitespace, character_set["space"])
    .str.replace(regex_unknown, '')
    .str.strip()
)

In [ ]:
congress_tweets.to_pickle('../Data/Processed/congress_cleaned_processed.pkl')